# Query traces from Application Insights
Inspired by https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/online-evaluation?tabs=windows

## Documentation

https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-local-sdk?tabs=python

https://learn.microsoft.com/en-us/azure/azure-monitor/app/opentelemetry-enable?tabs=aspnetcore

## Setup

### Common packages

In [1]:
import os
import dotenv
from pathlib import Path

### Global settings

In [2]:
# Global variables
PRIVATE = False
DATA_DIR = Path("data")
TMP_DIR = Path("tmp")

### Load environment variables

In [3]:
# Import override environment variables from .env file
# or from private.env file if PRIVATE is True
dotenv.load_dotenv('.env' if not PRIVATE else 'private.env', override=True)

True

### Azure credentials

In [4]:
# https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

### Get Azure AI Foundry project client

In [5]:
from azure.ai.projects import AIProjectClient

# Create an Azure AI Client from a connection string. Available on Azure AI project Overview page.
# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.aiprojectclient?view=azure-python-preview
project_client = AIProjectClient.from_connection_string(
    os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING"), credential=credential)
                                 

### Check Application insights is enabled for this Azure AI Foundry project

In [6]:
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("WARNING: Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")


### Get AppId from Application insights connection string

In [7]:
appId=dict(item.split("=", 1) for item in application_insights_connection_string.split(";") if "=" in item)['ApplicationId']

## Fetch tracing data from Application Insights

### Define kusto Query

In [8]:
# https://learn.microsoft.com/en-us/python/api/azure-monitor-opentelemetry/azure.monitor.opentelemetry?view=azure-python    
# https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/online-evaluation?tabs=windows

# Kusto Query Language (KQL) query to query data from Application Insights resource
# This query is compatible with data logged by the Azure AI Inferencing Tracing SDK (linked in documentation)
# You can modify it depending on your data schema
# The KQL query must output these required columns: operation_ID, operation_ParentID, and gen_ai_response_id
# You can choose which other columns to output as required by the evaluators you are using

# load string from file "traces_query.kql" into variable KUSTO_QUERY
with open("traces_query.kql", "r") as file:
    KUSTO_QUERY = file.read()

### Wrap Azure credentials for compatibility with Application Insights SDK

In [9]:
from msrest.authentication import BasicTokenAuthentication

# Wrap Azure credentials for compatibility with Application Insights SDK
class CredentialWrapper(BasicTokenAuthentication):
    def __init__(self, credential=None, resource_id="https://api.applicationinsights.io/.default"):
        super(CredentialWrapper, self).__init__(None)
        self.credential = credential
        self.resource_id = resource_id

    def set_token(self):
        token = self.credential.get_token(self.resource_id)
        self.token = {"access_token": token.token}

    def signed_session(self, session=None):
        self.set_token()
        return super(CredentialWrapper, self).signed_session(session)


### Create application insights client

In [10]:
from azure.applicationinsights import ApplicationInsightsDataClient
appi_client = ApplicationInsightsDataClient(credentials=CredentialWrapper(credential=credential))

### Query traces from Application Insights

In [11]:
from azure.applicationinsights.models import QueryBody
from tabulate import tabulate


# appId = You can find this in the Azure Portal under the "API Access" section of your Application Insights resource.
response= appi_client.query.execute(app_id=appId, body=QueryBody(query=KUSTO_QUERY, timespan="PT24H"))
# Process and display the results

if response.tables:
    for table in response.tables:
        # Extract column names
        headers = [col.name for col in table.columns]
        
        # Extract rows
        rows = table.rows
        
        # Display the table using tabulate
        print(tabulate(rows, headers=headers, tablefmt="grid"))
else:
    print("No data found.")

+----------------------------------------------+---------------------------------------------------------+-----------------------------------------------------+----------------------------------+----------------------------------+----------------------------------------+
| Input                                        | System                                                  | Output                                              | operation_Id                     | operation_ParentId               | gen_ai_response_id                     |
+==============================================+=========================================================+=====================================================+==================================+==================================+========================================+
| How many languages are in the Europe?        | You're a chatbot that only responds in whimsical rhyme! |                                                     | b065a892690cf2592683b6d

## Save traces as json

In [12]:
import json
from datetime import datetime
            
def serialize_to_jsonl(response, file_prefix="traces"):
    # Assuming `response.tables` contains the tables from the Application Insights query
    if response.tables:
        for table in response.tables:
            # Extract column names
            headers = [col.name for col in table.columns]
            
            # Extract rows
            rows = table.rows
            
            # Serialize each row as a JSON object and write to a JSONL file
            
            # Generate a human-readable timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            output_file = f"{file_prefix}_{table.name}_{timestamp}.jsonl"
            with open(output_file, "w") as jsonl_file:
                for row in rows:
                    json_line = {headers[i]: row[i] for i in range(len(headers))}
                    jsonl_file.write(json.dumps(json_line) + "\n")
            
            print(f"Serialized table {table.name} to {output_file}")
    else:
        print("No data found.")

serialize_to_jsonl(response, file_prefix=TMP_DIR / "from_appi")

Serialized table PrimaryResult to tmp\from_appi_PrimaryResult_2025-06-23_16-47-16.jsonl
